In [2]:
import advent
advent.scrape(2015, 19)
data = advent.get_lines_doublenewline(19)
rules_raw, molecule = data
molecule: str = molecule[0]
MAXLEN = len(molecule)
rules = []
for rule in rules_raw:
    rules.append(rule.split(' => '))

In [12]:
# I turned this into a function for part 2, but it's really just the same what I wrote for part 1 just wrapped in a function
# I also added a tuple to possibilities instead of the element, but this has no impact on part 2
def adjacent(molecule: str):
    if len(molecule) > MAXLEN: return []
    possibilities = set([])
    for left, right in rules:
        if left in molecule:
            indexes = [i for i in range(len(molecule)) if molecule.startswith(left, i)]
            for ix in indexes:
                possibilities.add((molecule[:ix] + molecule[ix:].replace(left, right, 1), 1))
    return possibilities
print(len(adjacent(molecule)))


576


In [13]:
# Could try dynamic programming but dijkstra seems more fun :D
from advent.maze import solve_maze

# This didn't end up working
#solve_maze('e', lambda x: x==molecule, adjacent)


In [14]:
def adjacent_reverse(molecule: str):
    for right, left in rules:
        if left in molecule:
            for ix in range(len(molecule)):
                if molecule.startswith(left, ix):
                    yield (molecule[:ix] + molecule[ix:].replace(left, right, 1), 1)

print(list(adjacent_reverse("SiRnFYFArCaF")))

# too slow
#solve_maze(molecule, lambda x: x == 'e', adjacent_reverse)

# At this point I did some googling and found this is a context-free grammar (duh), and you can use nltk

[('CaCaF', 1), ('SiRnFYFArF', 1)]


In [ ]:
!pip install nltk

In [10]:
import nltk
import re
nltk.download('punkt')

# Lets get our symbols
terminals = set([])
nonterminals = set([])

for rule in rules:
    nonterminals.add(rule[0])

molecule_grammar = re.findall(r'[A-Z][a-z]*', molecule)

grammar = "S -> e\n"
# Add dummy start rule 'S -> e' so NLTK understands us. remember to subtract 1 from the answer!
for rule in rules:
    outputs = re.findall(r'[A-Z][a-z]*', rule[1])
    for o in outputs:
        if o not in nonterminals: terminals.add(o)
    os = ' '.join(o for o in outputs )
    grammar += f"{rule[0]} -> {os}" + "\n"
for c in set(re.findall(r'[A-Z][a-z]*', molecule)):
    grammar += f"{c} -> '{c}'\n"

nltk_grammar = nltk.CFG.fromstring(grammar)
print(nltk_grammar.start())

S


[nltk_data] Downloading package punkt to /Users/fenno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
molecule_grammart = nltk.word_tokenize(' '.join(re.findall(r'[A-Z][a-z]*', molecule)))

#parser = nltk.ChartParser(nltk_grammar)
#trees = list(parser.parse(molecule_grammart))
#print(trees[0].height() - 1)
# This still took like 20 minutes

# apparently the CYK algorithm is supposed to run in O(n^3 * |G|)
# Where n is length of string and |G| is number of rules, so there's no way NLTK is using that
# because that sounds like a really low complexity
# Let's try implementing cyk ourselves

In [60]:
# for cyk we need chompsky normal form
# follow procedure described here: https://en.wikipedia.org/wiki/Chomsky_normal_form
# We only need to do TERM and BIN

nts = ['e'] + list((terminals | nonterminals) - set(['e']))
ts = [f"t{c}" for c in nts if c != 'e']
cnf_rules = []
cnt = 1
for rule in rules:
    outputs = re.findall(r'[A-Z][a-z]*', rule[1])
    assert len(outputs) > 1
    leftsymbol = rule[0]
    while len(outputs) > 2:
        cnf_rules.append((leftsymbol, [outputs[0], f"X{cnt}"]))
        leftsymbol, outputs, cnt = f"X{cnt}", outputs[1:], cnt+1
    cnf_rules.append((leftsymbol, outputs))
for t in ts:
    cnf_rules.append((t[1:], [t]))

inputstr = ['t'+s for s in molecule_grammart]

print(cnf_rules)
print(nts)
print(ts)
print(inputstr)
print(len(cnf_rules))
assert nts[0] == 'e'

[('Al', ['Th', 'F']), ('Al', ['Th', 'X1']), ('X1', ['Rn', 'X2']), ('X2', ['F', 'Ar']), ('B', ['B', 'Ca']), ('B', ['Ti', 'B']), ('B', ['Ti', 'X3']), ('X3', ['Rn', 'X4']), ('X4', ['F', 'Ar']), ('Ca', ['Ca', 'Ca']), ('Ca', ['P', 'B']), ('Ca', ['P', 'X5']), ('X5', ['Rn', 'X6']), ('X6', ['F', 'Ar']), ('Ca', ['Si', 'X7']), ('X7', ['Rn', 'X8']), ('X8', ['F', 'X9']), ('X9', ['Y', 'X10']), ('X10', ['F', 'Ar']), ('Ca', ['Si', 'X11']), ('X11', ['Rn', 'X12']), ('X12', ['Mg', 'Ar']), ('Ca', ['Si', 'Th']), ('F', ['Ca', 'F']), ('F', ['P', 'Mg']), ('F', ['Si', 'Al']), ('H', ['C', 'X13']), ('X13', ['Rn', 'X14']), ('X14', ['Al', 'Ar']), ('H', ['C', 'X15']), ('X15', ['Rn', 'X16']), ('X16', ['F', 'X17']), ('X17', ['Y', 'X18']), ('X18', ['F', 'X19']), ('X19', ['Y', 'X20']), ('X20', ['F', 'Ar']), ('H', ['C', 'X21']), ('X21', ['Rn', 'X22']), ('X22', ['F', 'X23']), ('X23', ['Y', 'X24']), ('X24', ['Mg', 'Ar']), ('H', ['C', 'X25']), ('X25', ['Rn', 'X26']), ('X26', ['Mg', 'X27']), ('X27', ['Y', 'X28']), ('X28', 

In [62]:
# https://en.wikipedia.org/wiki/CYK_algorithm
from collections import defaultdict
from functools import cache
n = len(inputstr)
r = len(nts) + cnt - 1  # nts plus X1..X_cnt-1
P = defaultdict(bool) # (1..n, 1..n, 1..r) -> bool, default=False
back = defaultdict(list) # (1..n, 1..n, 1..r) -> bool, default=[]

@cache
def number_nonterminal(t: str) -> int:
    # output ranges from 1 to r inclusive
    if t in nts: return nts.index(t) + 1
    return int(t[1:]) + len(nts)

for s in range(1, n+1):
    terminal_s = inputstr[s-1]
    for rule in cnf_rules:
        assert isinstance(rule[1], list)
        if len(rule[1]) == 1 and rule[1][0] == terminal_s:
            v = number_nonterminal(rule[0])
            P[(1,s,v)] = True
assert P[(1, 1, number_nonterminal('O'))] # sanity check, inputstr[1-1] is O
assert P[(1, 2, number_nonterminal('Rn'))]
assert number_nonterminal('e') == 1

from tqdm import tqdm

for l in tqdm(range(2, n+1)): # length of span
    for s in range(1, n-l+2):  # start of span
        for p in range(1, l):  # partition of span
            for rule in cnf_rules:
                if len(rule[1]) == 1: continue  # only nonterminal rules
                a = number_nonterminal(rule[0])
                b = number_nonterminal(rule[1][0])
                c = number_nonterminal(rule[1][1])
                if P[(p,s,b)] and P[(l-p,s+p,c)]:
                    P[(l,s,a)] = True
                    back[(l,s,a)] += [(p,b,c)]
# takes like 2.5 minutes
if P[(n,1,1)]:
    print("Success! Time to start inspecting back") 

100%|██████████| 291/291 [02:40<00:00,  1.81it/s]

Success! Time to start inspecting back


In [74]:
# n, n, r = length, start, terminal
print(n)
print(back[(n, 1, 1)]) # length n, start 1, terminal 1 (e)
# The elements of this are (p, b, c) meaning: partition input I at p
# Then everything left of that is terminal b, right of that is terminal c
# Example: (7, 12, 6), meaning first 7 chars is terminal 12(H), 8 and rest is terminal 6(F)
# Now we want to backtrace (7, 12, 6), meaning we want to:
# - fit the first 7 characters of I with 12
# - fit the remaining characters of I with 6
print(back[(7, 1, 12)]) # only one way to do it
print(back[(n-7, 1+7, 6)]) # e.g. (3, 14, 6) meaning we split here at 3

292
[(7, 12, 6), (8, 12, 6), (9, 12, 6), (10, 12, 6), (11, 7, 15), (12, 12, 6), (13, 12, 6), (14, 12, 6), (15, 7, 15), (16, 12, 6), (17, 12, 6), (18, 12, 6), (19, 2, 4), (20, 12, 6), (27, 12, 6), (28, 12, 6), (29, 12, 6), (30, 7, 15), (31, 12, 6), (32, 12, 6), (33, 12, 6), (34, 7, 15), (35, 12, 6), (36, 12, 6), (37, 12, 6), (38, 12, 6), (39, 12, 6), (40, 12, 6), (41, 12, 6), (47, 12, 6), (51, 12, 6), (52, 12, 6), (64, 12, 6), (96, 12, 6), (97, 2, 4), (101, 12, 6), (111, 12, 6), (112, 12, 6), (152, 12, 6), (153, 7, 15), (154, 12, 6), (155, 7, 15), (156, 12, 6), (157, 12, 6), (158, 12, 6), (162, 12, 6), (163, 12, 6), (164, 12, 6), (168, 2, 4), (169, 2, 4), (170, 12, 6), (171, 2, 4), (177, 12, 6), (178, 12, 6), (179, 2, 4), (183, 12, 6), (184, 2, 4), (185, 12, 6), (186, 2, 4), (187, 12, 6), (188, 12, 6), (189, 12, 6), (190, 7, 15), (191, 12, 6), (192, 12, 6), (193, 2, 4), (194, 12, 6), (195, 7, 15), (196, 12, 6), (200, 12, 6), (201, 7, 15), (202, 12, 6), (203, 12, 6), (204, 7, 15), (205, 

In [ ]:
@cache
def traverse_back(length, partition, nt):
    #print(length, partition, nt)
    if length == 1: return 0 # no substitutions needed
    if not back[(length, partition, nt)]:
        print((length, partition, nt))
        return 9999999998  # Not sure but this happens apparently
    result = 99999999999997
    for p, b, c in back[(length, partition, nt)]:
        subs = traverse_back(p, partition, b) + \
               traverse_back(length-p, partition+p, c) + \
               (1 if max(b, c) <= 17 else 0)
        if subs < result:
            result=subs
    return result

# Well this gives 291, which is really trivial: we start with 1 symbol,
# each step adds 1 symbol, meaning we need 291 steps to get to n=292 symbols.
print(traverse_back(n, 1, 1))
# I then changed it to '(1 if max(b, c) <= 17 else 0)' (was just 1 before)
# Meaning we don't count any rules like A -> B Xi as an actual substitution

# And it gave 207. and to be honest at this point I was so deep in the weeds,
# if this was incorrect I would have given up (and tried again at a later date)
# But it was correct!

207


In [91]:
# Just to help me understand a bit what was going on
for i in nts:
    print(f"{i}: {number_nonterminal(i)}")
print("X1: 18")
print(f"X48: {18+47}")

e: 1
O: 2
Th: 3
Mg: 4
Y: 5
F: 6
N: 7
B: 8
Ti: 9
P: 10
C: 11
H: 12
Ar: 13
Ca: 14
Al: 15
Rn: 16
Si: 17
X1: 18
X48: 65
